In [1]:
import numpy as np
import networkx as nx
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import scipy
import time

sys.path.append('./src')
from generative_graph_models import initialize_sparse_L, normalize_sparse_L
from algorithms_optimal import cvx_optimizer
from utils import set_mkl_threads, assert_results
from generative_opinions_models import define_set_opinions, standardize

set_mkl_threads(16)



0

## Graph initialization

In [2]:
from sklearn.preprocessing import normalize

n = 50
A_true = np.random.choice(3, size=(n, n))
np.fill_diagonal(A_true, 0.)
A_true = normalize(A_true, axis=1, norm='l1')

I = np.identity(n)
z_eq = np.random.uniform(size=(n, 1))
s = (I + I-A_true) @ z_eq
assert np.all((z_eq - np.linalg.inv(I + np.diag(A_true.sum(axis=1)) - A_true) @ s) < 1e-9)

In [8]:
D_in = np.diag(A_true.sum(axis=0))
s.T@s + s.T@D_in@s - 2*s.T@A_true@s

array([[33.67495908]])

In [10]:
r = 0
for i in range(n):
    for j in range(n):
        r += A_true[i, j] * (s[i]- s[j])**2
r

array([33.67495908])

array([[33.8096462]])

### Checking equilibrium formulas

# Gradients

In [915]:
def fAndG(A, s):
    s = s.flatten()

    T_0 = (2 * np.eye(A_cols, A_cols))
    T_1 = np.linalg.inv((T_0 + -A))
    t_2 = (T_1).dot(s)
    t_3 = (np.linalg.inv((T_0 - A))).dot(s)
    t_4 = (T_1).dot(((np.diag((A).dot(np.ones(A_cols))) - np.eye(A_cols, A_cols))).dot(t_3))
    t_5 = (s).dot(T_1)
    functionValue = ((s).dot(t_2) + (0.5 * (s).dot(t_4)))
    gradient = (((np.outer(t_2, t_5) + (0.5 * np.outer(t_4, t_5))) + (0.5 * np.outer(np.ones(A_rows), (t_5 * t_5)))) + (0.5 * np.outer((T_1).dot(((np.diag((np.ones(A_cols)).dot(A)) - np.eye(A_cols, A_cols))).dot(t_3)), t_5)))

    return gradient


def fAndG4(A, s):
    s = s.flatten()

    T_0 = (2 * np.eye(A_cols, A_cols))
    T_1 = np.linalg.inv((T_0 + -A.T))
    t_2 = (T_1).dot(s)
    t_3 = (np.linalg.inv((T_0 - A))).dot(s)
    t_4 = (T_1).dot(((np.diag((A.T).dot(np.ones(A_cols))) - np.eye(A_cols, A_cols))).dot(t_3))
    t_5 = (s).dot(T_1)
    functionValue = ((s).dot(t_2) + (0.5 * (s).dot(t_4)))
    gradient = (((np.outer(t_2, t_5) + (0.5 * np.outer(t_4, t_5))) + (0.5 * np.outer(np.ones(A_rows), (t_5 * t_5)))) + (0.5 * np.outer((T_1).dot(((np.diag((np.ones(A_cols)).dot(A)) - np.eye(A_cols, A_cols))).dot(t_3)), t_5)))

    return functionValue, gradient

def fAndG2(A, I, s):
    """s'*(inv(2*I-A))'*s + 0.5*s'*(inv(2*I-A))'*( diag(A'*vector(1)) - I )*inv(2*I-A)*s"""
    s = s.flatten()

    T_0 = np.linalg.inv(((2 * I) - A))
    t_1 = (T_0.T).dot(s)
    t_2 = (T_0).dot(s)
    t_3 = (T_0.T).dot(((np.diag((A.T).dot(np.ones(I_rows))) - I)).dot(t_2))
    functionValue = ((s).dot(t_1) + (0.5 * (s).dot(t_3)))
    gradient = (((np.outer(t_1, t_2) + (0.5 * np.outer(t_3, t_2))) + (0.5 * np.outer(np.ones(A_rows), (t_2 * t_2)))) + (0.5 * np.outer((T_0.T).dot(((np.diag((np.ones(I_rows)).dot(A)) + -I.T)).dot(t_2)), t_2)))

    return functionValue, gradient



def fAndG3(A, I, s):
    s = s.flatten()

    T_0 = (2 * I)
    T_1 = np.linalg.inv((T_0 + -A.T))
    t_2 = (T_1).dot(s)
    t_3 = (T_1).dot(((np.diag((A.T).dot(np.ones(I_rows))) - I)).dot((np.linalg.inv((T_0 - A))).dot(s)))
    t_4 = (s).dot(T_1)
    functionValue = ((s).dot(t_2) + (0.5 * (s).dot(t_3)))
    gradient = (((np.outer(t_2, t_4) + (0.5 * np.outer(t_3, t_4))) + (0.5 * np.outer(np.ones(A_rows), (t_4 * t_4)))) + (0.5 * np.outer((T_1).dot(((np.diag((np.ones(I_rows)).dot(A)) - I)).dot((np.linalg.inv((T_0.T - A))).dot(s))), t_4)))

    return functionValue, gradient

In [916]:
def fAndG2(A, I, s):
    """s'*(inv(2*I-A))'*s + 0.5*s'*(inv(2*I-A))'*( diag(A'*vector(1)) - I )*inv(2*I-A)*s"""
    s = s.flatten()
    
    assert isinstance(A, np.ndarray)
    dim = A.shape
    assert len(dim) == 2
    A_rows = dim[0]
    A_cols = dim[1]
    assert isinstance(I, np.ndarray)
    dim = I.shape
    assert len(dim) == 2
    I_rows = dim[0]
    I_cols = dim[1]
    assert isinstance(s, np.ndarray)
    dim = s.shape
    assert len(dim) == 1
    s_rows = dim[0]
    assert s_rows == I_cols == I_rows == A_rows == A_cols


    T_0 = np.linalg.inv(((2 * I) - A))
    t_1 = (T_0.T).dot(s)
    t_2 = (T_0).dot(s)
    t_3 = (T_0.T).dot(((np.diag((A.T).dot(np.ones(I_rows))) - I)).dot(t_2))
    functionValue = ((s).dot(t_1) + (0.5 * (s).dot(t_3)))
    gradient = (((np.outer(t_1, t_2) + (0.5 * np.outer(t_3, t_2))) + (0.5 * np.outer(np.ones(A_rows), (t_2 * t_2)))) + (0.5 * np.outer((T_0.T).dot(((np.diag((np.ones(I_rows)).dot(A)) + -I.T)).dot(t_2)), t_2)))

    return gradient

In [918]:
# 2. we simplify gradient formula and test it
def fAndG2_simpler(A, I, s):
    Ω = np.linalg.inv(((2 * I) - A))
    grad = Ω.T @ s
    grad = grad @ s.T @ Ω.T
    grad = grad + Ω.T @ (np.diag(A.sum(axis=0)) - I)@Ω@s@s.T@Ω.T
    grad = grad + .5* np.ones((len(s),1)) @ (np.multiply(Ω@s,Ω@s)).T
    return grad

_grad1 = fAndG2(A_true, I, s)
_grad2 = fAndG2_simpler(A_true, I, s)
np.all(np.absolute(_grad1 - _grad2)<1e-6)

True

True

In [969]:
# 3. we sparsify gradient formula and test it
def fAndG2_fast(A, s, Is, Js, ε=1e-6):
    """Returns gradient data array"""

    I = scipy.sparse.identity(len(s))
    
    z_1, _ = scipy.sparse.linalg.bicg(2*I-A.T, s, tol=ε)
    z_1 = z_1.reshape((len(s), 1))
    z_2, _ = scipy.sparse.linalg.bicg(2*I-A, s, tol=ε)
    z_2 = z_2.reshape((len(s), 1))
    grad = np.multiply(z_1[Is], z_2[Js]).flatten()
    
    z_3, _ =  scipy.sparse.linalg.bicg(2*I-A.T, (np.diag(A.sum(axis=0)) - I) @ z_2, tol=ε)
    z_3 = z_3.reshape((len(s), 1))
    grad = grad + np.multiply(z_3[Is], z_2[Js]).flatten()
    grad = grad + .5 * (z_2**2)[Js].flatten()
    return grad, z_2


_grad2 = fAndG2_simpler(A_true, I, s)
_grad2[A_true==0] = 0.

Is, Js = np.nonzero(A_true)
_grad3_data, z_2 = fAndG2_fast(A_true, s, Is, Js, 1e-9)
_grad3 = scipy.sparse.csr_matrix(A_true)
_grad3.data = _grad3_data
np.all(np.absolute(_grad2 - _grad3.todense()) < 1e-9)


True

# Experiments

In [970]:
# 1. Upper bound

def obj_f(z, A):
    return (0.5*z.T @ (I + np.diag(np.array(A.sum(axis=0)).flatten()) - (A+A.T)) @ z + z.T @ z).item()
UB = obj_f(z_eq, A_true)
UB

15.94235019390393

In [979]:
np.diag(np.array(scipy.sparse.csr_matrix(A_true).sum(axis=0)).flatten())

array([[1.02939723, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.12582357, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.06834897, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.66196242, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.19897031,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.13350693]])

In [971]:
# 2. Baseline
A_oppo_heur = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        if A_true[i, j] > 0:
            A_oppo_heur[i, j] = (s[i]-s[j])**2
A_oppo_heur = normalize(A_oppo_heur, axis=1, norm='l1')
z_oppo_eq = np.linalg.inv(2*I - A_oppo_heur) @ s
obj_f(z_oppo_eq, A_oppo_heur)/UB

1.1676515369426685

In [972]:
z, _ = scipy.sparse.linalg.bicg(2*I-A_true, s, tol=1e-3)

In [973]:
A = scipy.sparse.csr_matrix(A_true)
Is, Js = np.nonzero(A)
obj_prev = np.inf

for i in range(100):
    print(obj, obj_prev)
    _grad3_data, z_2 = fAndG2_fast(A, s, Is, Js, 1e-9)
    
    A.data = A.data - .00002 * _grad3_data
    A.data[A.data<0] = 0
    #print(A.todense(),"\n",  A_true)
    A = normalize(A, axis=1, norm='l1')


    z_eq_curr = np.linalg.inv(2*I - A.todense()) @ s
    obj = obj_f(z_eq_curr, np.array(A.todense()))
    
    if obj <= 0 or np.isnan(obj) or obj > obj_prev:
        print("break", obj <= 0, np.isnan(obj), obj >= obj_prev, obj_prev, obj)
        break
    obj_prev = obj
    print(f"{i}, obj/UB={obj/UB*100:.3f}, {np.min(grad):.1f} {np.max(grad):.1f} {np.mean(grad):.1f}")

15.896678439795924 inf
0, obj/UB=99.714, -0.0 1.0 0.3
15.896678439795924 15.896678439795924
1, obj/UB=99.428, -0.0 1.0 0.3
15.851127759632478 15.851127759632478
2, obj/UB=99.143, -0.0 1.0 0.3
15.80570059122595 15.80570059122595
3, obj/UB=98.859, -0.0 1.0 0.3
15.76039932879457 15.76039932879457
4, obj/UB=98.575, -0.0 1.0 0.3
15.715226322981103 15.715226322981103
5, obj/UB=98.293, -0.0 1.0 0.3
15.670183880883283 15.670183880883283
6, obj/UB=98.011, -0.0 1.0 0.3
15.625274266112495 15.625274266112495
7, obj/UB=97.730, -0.0 1.0 0.3
15.580499698863006 15.580499698863006
8, obj/UB=97.450, -0.0 1.0 0.3
15.535862356003463 15.535862356003463
9, obj/UB=97.171, -0.0 1.0 0.3
15.491364371184497 15.491364371184497
10, obj/UB=96.893, -0.0 1.0 0.3
15.447007834962971 15.447007834962971
11, obj/UB=96.616, -0.0 1.0 0.3
15.402794794942377 15.402794794942377
12, obj/UB=96.339, -0.0 1.0 0.3
15.35872725592866 15.35872725592866
13, obj/UB=96.064, -0.0 1.0 0.3
15.314807180101052 15.314807180101052
14, obj/UB=95

In [870]:
obj

106.91032782629085

----

In [31]:
# CHECK MEMORY USAGE
import subprocess
import re
import pandas as pd

def mem():

    cmd = subprocess.check_output("""top -bn 1 -o RES | grep -E "^( |[0-9])" | awk '{ printf("%-8s  %-8s  %-8s\\n", $2, $9, $10); }'""",
                              shell=True, universal_newlines=True)
    cmd = cmd.split('\n')

    data = []
    for line in cmd[1:]:
        if line == '':
            continue
        user, pcpu, pmem = re.split(r'\s+', line.strip())
        #print(pmem)
        #pmem = pmem.split(',')[0] +'.'+ pmem.split(',')
        data.append([user, pcpu, float(pmem)])

    columns = re.split(r'\s+', cmd[0].strip())
    df = pd.DataFrame(data, columns=columns).groupby('USER').sum().sort_values('%MEM', ascending=False)
    df = df[(df.index != 'root') & (df['%MEM'] >= 0.5)]

    msg =  "*Top users by memory usage:*\n```" + df.to_string() + '```'
    return msg

print(mem())

*Top users by memory usage:*
```          %MEM
USER          
apotra    26.3
guidoba+  17.1
spoetto    7.2
tlancia+   6.3
alberto+   5.8
andreac+   2.9
yanxia     2.7
corrado    1.8
fcinus     1.7
leitao     1.3
jacopo.+   0.6```


/tmp/ipykernel_2083992/4021794878.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = pd.DataFrame(data, columns=columns).groupby('USER').sum().sort_values('%MEM', ascending=False)
